In [1]:
import pandas as pd
import numpy as np

tradeData = pd.read_excel(r'C:\Users\jlho\ArcGIS\TestTradeData.xlsx')
capitals = pd.read_excel(r'C:\Users\jlho\ArcGIS\Capitals.xlsx')

#print(tradeData)

#selection = tradeData.loc[:,'Commodity Code']
#print(selection)

#select = tradeData.iloc[[1,3],2]
#print(select)

#Selection = tradeData['Commodity Code']
#print(Selection)

print(tradeData)
print(capitals)

   Commodity Code                              Commodity Description    Flow  \
0          2931.9  Separate chemically defined organo-inorganic c...  Export   
1          3002.0  Human blood; animal blood prepared for therape...  Export   
2          3002.0  Human blood; animal blood prepared for therape...  Export   
3          3002.0  Human blood; animal blood prepared for therape...  Export   
4          3603.0  Safety fuses; detonating fuses; percussion or ...  Export   
5          3603.0  Safety fuses; detonating fuses; percussion or ...  Export   
6          3603.0  Safety fuses; detonating fuses; percussion or ...  Export   
7          3603.0  Safety fuses; detonating fuses; percussion or ...  Export   

  Partner Iso2 Partner Iso3      Partner     Period  Quantity Kg  \
0           AU          AUS    Australia 2012-12-31        12.00   
1           NZ          NZL  New Zealand 2012-12-31      4863.00   
2           AU          AUS    Australia 2012-12-31         2.00   
3      

In [2]:
#I want to make a matrix that has the origin and destination and the two sets of coordinates. It does not matter if the name of the capital is included. 
#I also want to make a matrix that has the trade information. This could be in the same dataset. It could also be in a separate but parallel one. 

In [6]:
#Need to match up partner and reporter with data points. All will be oriented in origin -> destination format.
#Therefore, all will be [OriginLat, OriginLong, DestLat, DestLong]
#Must have if flow is export, then reporter is origin. If flow is import, then reporter is destination.
#this matrix contains the coordinate data for each origin -> destination pair. 

#Partner = tradeData.iloc[:,5]
#Reporter = tradeData.iloc[:,10]
import numpy as np
import pandas as pd

tradeData = pd.read_excel(r'C:\Users\jlho\ArcGIS\TestTradeData.xlsx')
capitals = pd.read_excel(r'C:\Users\jlho\ArcGIS\Capitals.xlsx')

latlong = np.zeros((len(tradeData), 4))
for i in range(0,len(tradeData)): #scanning each city in the trade data
    for j in range(0,len(capitals)): #scanning the city in the capital database for each trade data city
        if tradeData.iloc[i,5] == capitals.iloc[j,0]: #assign coordinates if the capital and trade data partner city line up. 
            partnerLat = capitals.iloc[j,2]
            partnerLong = capitals.iloc[j,3]
            #print('Partner: ',partnerLat, partnerLong)
        if tradeData.iloc[i,10] == capitals.iloc[j,0]: #assign coordinates if the capital and trade data reporter city line up. 
            reporterLat = capitals.iloc[j,2]
            reporterLong = capitals.iloc[j,3]
            #print('Reporter: ', reporterLat, reporterLong)
    if tradeData.iloc[i,2] == 'Export': #put coordinates in correct column depending on whether the goods are exported or imported. 
        row = [reporterLat, reporterLong, partnerLat, partnerLong]
    elif tradeData.iloc[i,2] == 'Import': 
        row = [partnerLat, partnerLong, reporterLat, reporterLong]
    latlong[i,:] = row

LatLong = pd.DataFrame(latlong)
LatLong.columns = ['Origin Lat','Origin Long','Dest Lat','Dest Long']
    #note: when importing to ArcGIS, Lat=y, Long=x. 


LatLong = LatLong.as_matrix(['Origin Lat','Origin Long','Dest Lat','Dest Long']) #pandas 0.18.1
#LatLong = LatLong.to_numpy() #pandas 1.2.4
#print(LatLong)

#LatLong.to_excel(r'C://Users//jlho//ArcGIS//latlong.xlsx', index=False)

structLatLong = np.core.records.fromarrays(LatLong.transpose(), np.dtype([('Origin Lat','>f8'),
                                                              ('Origin Long','>f8'),
                                                              ('Dest Lat','>f8'),
                                                              ('Dest Long','>f8')]))
print("structLatLong is: ", structLatLong)

#creating structured array to export to ArcGIS

import arcpy 
#convert array into geodatabase table
table = arcpy.da.NumPyArrayToTable(structLatLong, "//snl/home/jlho/Documents/ArcGIS/Default.gdb/coordtable")
#something is wrong with the coordtable. It is changing the values in structLatLong.
#create lines
coordtable = r'//snl/home/jlho/Documents/ArcGIS/Default.gdb/coordtable'
out_lines = r'//snl/home/jlho/Documents/ArcGIS/Default.gdb/supplyLines'
lines = arcpy.XYToLine_management(coordtable, out_lines,'Origin_Long','Origin_Lat','Dest_Long','Dest_Lat','GEODESIC')

            

structLatLong is:  [(-34.603684, -58.381559, -35.282  , 149.128684)
 (-34.603684, -58.381559, -41.28646, 174.776236)
 (-15.794229, -47.882166, -35.282  , 149.128684)
 (-15.794229, -47.882166, -41.28646, 174.776236)
 (-15.794229, -47.882166, -35.282  , 149.128684)
 (-33.44889 , -70.669265, -35.282  , 149.128684)
 (-33.44889 , -70.669265, -41.28646, 174.776236)
 (-12.046374, -77.042793, -41.28646, 174.776236)]


ModuleNotFoundError: No module named '_base'

In [15]:
recs = [['M', 64, 75.0],['F', 25, 60.0]]
dts = {'names': ('gender','age','weight'),
       'formats':('S1', np.uint8, np.float32)}
array = np.rec.fromrecords(recs, dtype=dts)
print(array)

[(b'M', 64, 75.) (b'F', 25, 60.)]


<ipython-input-15-bec18068ce65>:4: FutureWarning: fromrecords expected a list of tuples, may have received a list of lists instead. In the future that will raise an error
  array = np.rec.fromrecords(recs, dtype=dts)


In [3]:
latlong = np.zeros((2,3))
row = [1,2,3]

latlong[0,:] = row
print(latlong)

print(pd.__version__)

[[1. 2. 3.]
 [0. 0. 0.]]
1.2.4


In [40]:
#Dataframe modification for non-numerical data. 

#Partner = tradeData.iloc[:,5]
#Reporter = tradeData.iloc[:,10]
import numpy as np
import pandas as pd

tradeData = pd.read_excel(r'C:\Users\jlho\ArcGIS\TestTradeData.xlsx')
capitals = pd.read_excel(r'C:\Users\jlho\ArcGIS\Capitals.xlsx')

columnHeadings = {'Origin Lat':[],'Origin Long':[],'Dest Lat':[],'Dest Long':[],
                  'Strategic Commodity':[], 'Value':[],'Quantity (kg)':[]}
supplyData = pd.DataFrame(columnHeadings) 
for i in range(0,len(tradeData)): #scanning each city in the trade data
    Quantity = tradeData.iloc[i,7] 
    Strat_Comm = tradeData.iloc[i,11]
    Value = tradeData.iloc[i,12]
    for j in range(0,len(capitals)): #scanning the city in the capital database for each trade data city
        if tradeData.iloc[i,5] == capitals.iloc[j,0]: #assign coordinates if the capital and trade data partner city line up. 
            partnerLat = capitals.iloc[j,2]
            partnerLong = capitals.iloc[j,3]
        if tradeData.iloc[i,10] == capitals.iloc[j,0]: #assign coordinates if the capital and trade data reporter city line up. 
            reporterLat = capitals.iloc[j,2]
            reporterLong = capitals.iloc[j,3]
    if tradeData.iloc[i,2] == 'Export': #put coordinates in correct column depending on whether the goods are exported or imported. 
        row = [{'Dest Lat':reporterLat,'Dest Long':reporterLong, 'Origin Lat': partnerLat, 'Origin Long': partnerLong, 
               'Strategic Commodity':Strat_Comm, 'Value':Value, 'Quantity (kg)': Quantity}]
    elif tradeData.iloc[i,2] == 'Import': 
        row = [{'Origin Lat': partnerLat, 'Origin Long': partnerLong, 'Dest Lat':reporterLat,
               'Dest Long':reporterLong,'Strategic Commodity':Strat_Comm, 'Value':Value, 'Quantity (kg)': Quantity}]
    supplyData = supplyData.append(row)
print(supplyData)

npSupplyData = supplyData.to_numpy()
print(npSupplyData)
print(np.shape(npSupplyData))

#note: when importing to ArcGIS, Lat=y, Long=x. 

#supplyData.to_excel(r'C://Users//jlho//ArcGIS//extendedSupplyData.xlsx', index=False)

#creating structured array to export to ArcGIS

dts = np.dtype({'names': ['Origin Lat', 'Origin Long', 'Dest Lat', 'Dest Long',
                          'Strategic Commodity', 'Value', 'Quantity (kg)'], 
                'formats': ['f8', 'f8', 'f8', 'f8','S30','f8','f8']})
print("dict len: ", len(dts))
arcCoords = np.rec.fromrecords(npSupplyData, dtype=dts)
print('arcCoords: ', arcCoords)

import arcpy 
#convert array into geodatabase table
arcpy.da.NumPyArrayToTable(arcCoords, "c:/Users/jlho/ArcGIS/SupplyCoords")

   Origin Lat  Origin Long   Dest Lat  Dest Long         Strategic Commodity  \
0   -35.28200   149.128684 -34.603684 -58.381559     CWC scheduled chemicals   
0   -41.28646   174.776236 -34.603684 -58.381559  Micro-organisms and toxins   
0   -35.28200   149.128684 -15.794229 -47.882166  Micro-organisms and toxins   
0   -41.28646   174.776236 -15.794229 -47.882166  Micro-organisms and toxins   
0   -35.28200   149.128684 -15.794229 -47.882166                  Detonators   
0   -35.28200   149.128684 -33.448890 -70.669265                  Detonators   
0   -41.28646   174.776236 -33.448890 -70.669265                  Detonators   
0   -41.28646   174.776236 -12.046374 -77.042793                  Detonators   

        Value  Quantity (kg)  
0   145166.73          12.00  
0   379156.00        4863.00  
0     3981.00           2.00  
0    42266.09         133.11  
0    21856.70         259.87  
0  6826605.65      228881.59  
0    94676.12        2066.58  
0    81191.22         485.32  


<ipython-input-40-58290fd792b4>:48: FutureWarning: fromrecords expected a list of tuples, may have received a list of lists instead. In the future that will raise an error
  arcCoords = np.rec.fromrecords(npSupplyData, dtype=dts)


ModuleNotFoundError: No module named '_base'

In [4]:
import pandas as pd

data={'Name':['Karan','Rohit','Sahil','Aryan'],'Age':[23,22,21,24]}

df= pd.DataFrame(data)
newrow = [{'Name':'Julia','Age':21}]
df.append(newrow)



,Name,Age
0,Karan,23
1,Rohit,22
2,Sahil,21
3,Aryan,24
0,Julia,21
